In this notebook we will use the nn class **Sequential** to model a CNN

We will use the single target network

# Import

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

In [2]:
import load_script

In [3]:
%load_ext autoreload
%autoreload 2

# Loading the dataset

In [4]:
train_dataset,target=load_script.load_dataset(train=True,single_target=True)

In [5]:
test_dataset,test_target=load_script.load_dataset(train=False,single_target=True)

# Defining the network

### Single output

In [6]:
class SingleOutput(nn.Module):
    def __init__(self):
        super(SingleOutput,self).__init__()

        self.conv1=nn.Conv1d(28,14,5)
        self.full1=nn.Linear(14*46,100)
        self.full2=nn.Linear(100,1)
        
    def forward(self,x):
        x=F.max_pool1d(F.relu(self.conv1(x)),1)
        x=x.view(-1)
        x=F.relu(self.full1(x.view(-1,14*46)))
        x=F.sigmoid(self.full2(x))
        
        return x

# Training

In [14]:
net=SingleOutput()

optimizer = optim.SGD(net.parameters(), lr=0.001)
criterion=nn.MSELoss()
mini_batch_size=79

for epoch in range(1001):
    
    total_loss=0
    output_target=torch.zeros(target.shape[0])
    for b in range(0,train_dataset.shape[0],mini_batch_size):
        
        
        train_element=train_dataset.narrow(0,b,mini_batch_size)
#         train_element=train_element.view(1,28,-1)
        
        target_element=target.narrow(0,b,mini_batch_size)
        
        optimizer.zero_grad()
        
#         print(train_element.shape)
        out=net(train_element)
#         print(out)
        output_target[b:b+mini_batch_size]=(out>0.5).data
#         print(out)
#         print(target_element)
        loss=criterion(out,target_element)
        loss.backward()
        optimizer.step()
        total_loss+=loss.data[0]
        
    output_test=torch.zeros(test_target.shape[0])
    out=net(test_dataset)
#         print(out)
    output_test=(out>0.5).data.type(torch.FloatTensor)
        
    
#     print(type(output_test))
        
    error_train=np.sum(list(output_target==target.data))/target.shape[0]
    error_test=np.sum(list(output_test[:,0]==test_target.data))/test_target.shape[0]
    if epoch%10==0:
        print('Epoch:',epoch,'Loss:',total_loss,'Correct:',str(error_train*100)[:5]+"%",
             'Correct test:',str(error_test*100)[:5]+"%")

Epoch: 0 Loss: 1.1961029171943665 Correct: 50.0% Correct test: 47.0%
Epoch: 10 Loss: 0.9653738290071487 Correct: 63.60% Correct test: 51.0%
Epoch: 20 Loss: 0.8787717968225479 Correct: 68.03% Correct test: 51.0%
Epoch: 30 Loss: 0.8168325871229172 Correct: 71.20% Correct test: 50.0%
Epoch: 40 Loss: 0.765731155872345 Correct: 73.10% Correct test: 53.0%
Epoch: 50 Loss: 0.721690908074379 Correct: 75.63% Correct test: 54.0%
Epoch: 60 Loss: 0.6846473664045334 Correct: 77.84% Correct test: 54.0%
Epoch: 70 Loss: 0.6523829400539398 Correct: 79.43% Correct test: 54.0%
Epoch: 80 Loss: 0.6226917058229446 Correct: 80.69% Correct test: 55.0%
Epoch: 90 Loss: 0.5947512835264206 Correct: 82.91% Correct test: 56.0%
Epoch: 100 Loss: 0.5691540762782097 Correct: 84.17% Correct test: 56.0%
Epoch: 110 Loss: 0.5452124699950218 Correct: 85.12% Correct test: 56.0%
Epoch: 120 Loss: 0.5195794776082039 Correct: 87.34% Correct test: 58.0%
Epoch: 130 Loss: 0.49239061027765274 Correct: 88.29% Correct test: 56.0%
Epoch